<a href="https://colab.research.google.com/github/Ekimkuznetsov/Data_from_JSON/blob/main/%D0%9A%D0%BE%D0%BF%D1%96%D1%8F_%D0%B7%D0%B0%D0%BF%D0%B8%D1%81%D0%BD%D0%B8%D0%BA%D0%B0_%22document_processing_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Document Processing with Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/document_processing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fdocument-processing%2Fdocument_processing.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>       
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/document_processing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/document-processing/document_processing.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Holt Skinner](https://github.com/holtskinner), [Drew Gillson](https://github.com/drewgillson) |

## Overview

In today's information-driven world, the volume of digital documents generated daily is staggering. From emails and reports to legal contracts and scientific papers, businesses and individuals alike are inundated with vast amounts of textual data. Extracting meaningful insights from these documents efficiently and accurately has become a paramount challenge.

Document processing involves a range of tasks, including text extraction, classification, summarization, and translation, among others. Traditional methods often rely on rule-based algorithms or statistical models, which may struggle with the nuances and complexities of natural language.

Generative AI offers a promising alternative to understand, generate, and manipulate text using natural language prompting. Gemini on Vertex AI allows these models to be used in a scalable manner through:

- [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) in the Cloud Console
- [Vertex AI REST API](https://cloud.google.com/vertex-ai/docs/reference/rest)
- [Vertex AI SDK for Python](https://cloud.google.com/vertex-ai/docs/python-sdk/use-vertex-ai-python-sdk-ref)
- [Other client libraries](https://cloud.google.com/vertex-ai/docs/start/client-libraries)

This notebook focuses on using the **Vertex AI SDK for Python** to call the Vertex AI Gemini API with the Gemini 1.5 model.

For more information, see the [Generative AI on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview) documentation.


### Objectives

In this tutorial, you will learn how to use the Vertex AI Gemini API with the Vertex AI SDK for Python to process PDF documents with the Gemini 1.5 (`gemini-1.5-pro-preview-0409`) model.

You will complete the following tasks:

- Install the Vertex AI SDK for Python
- Use the Vertex AI Gemini API to interact with Gemini 1.5 (`gemini-1.5-pro-preview-0409`) model:
  - Extract structured entities from an unstructured document
  - Classify document types
  - Combine classification and entity extraction into a single workflow
  - Summarize documents


### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.


## Getting Started


### Install Vertex AI SDK for Python


In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 4.4 MB/s eta 0:00:00


### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>



### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
# Define project information
PROJECT_ID = "mykhailo-project"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries


In [3]:
import json

from IPython.display import display, display_pdf, IFrame

from vertexai.generative_models import (
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Part,
)

List objects in the bucket

In [5]:
from google.cloud import storage

uris = []
def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""


    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    for blob in blobs:
        uris.append(f"gs://{bucket_name}/{blob.name}")
        print(f"gs://{bucket_name}/{blob.name}")
bucket_name = "anton_may_test"
list_blobs(bucket_name)

print(uris)


gs://anton_may_test/1119_001.jpg
gs://anton_may_test/1122_001.jpg
gs://anton_may_test/132-204_001.jpg
gs://anton_may_test/201_001.jpg
gs://anton_may_test/203.TIF
gs://anton_may_test/203_001.jpg
gs://anton_may_test/210_001.jpg
gs://anton_may_test/22.TIF
gs://anton_may_test/223_001.jpg
gs://anton_may_test/234.pdf
gs://anton_may_test/260.TIF
gs://anton_may_test/6546464.tif
gs://anton_may_test/output1_5.pdf
gs://anton_may_test/output_1.pdf
gs://anton_may_test/output_3.pdf
gs://anton_may_test/output_4.pdf
gs://anton_may_test/Запрос_204-ОГМет.tif
gs://anton_may_test/Запрос_производства_ОГМет-201.pdf
gs://anton_may_test/Повдомлення 1.pdf
['gs://anton_may_test/1119_001.jpg', 'gs://anton_may_test/1122_001.jpg', 'gs://anton_may_test/132-204_001.jpg', 'gs://anton_may_test/201_001.jpg', 'gs://anton_may_test/203.TIF', 'gs://anton_may_test/203_001.jpg', 'gs://anton_may_test/210_001.jpg', 'gs://anton_may_test/22.TIF', 'gs://anton_may_test/223_001.jpg', 'gs://anton_may_test/234.pdf', 'gs://anton_may_t

### Load the Gemini 1.5 model

Gemini 1.5 (`gemini-1.5-pro-preview-0409`) is a multimodal model that supports multimodal prompts. You can include text, image(s), and video in your prompt requests and get text or code responses.

In [6]:
model = GenerativeModel("gemini-1.5-pro-preview-0409")

generation_config = {
    "temperature": 0.0,
    "response_mime_type": "application/json",
}
safety_settings = {
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH
}

### Define helper functions

Define helper functions to load and parse files.

In [13]:
def get_file_bytes(file_path: str) -> bytes:
    with open(file_path, "rb") as file:
        return file.read()


def get_url_from_gcs(gcs_uri: str) -> str:
    # converts gcs uri to url for image display.
    return "https://storage.googleapis.com/" + gcs_uri.replace("gs://", "").replace(
        " ", "%20"
    )


def print_multimodal_prompt(contents: list):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if isinstance(content, Part):
            if content.inline_data:
                display_pdf(content.inline_data.data)
            elif content.file_data:
                display(
                    IFrame(
                        get_url_from_gcs(content.file_data.file_uri),
                        width=600,
                        height=300,
                    )
                )
        else:
            print(content)

# Send Google Cloud Storage Document to Google Cloud Storage
def process_document(
    prompt: str,
    file_uri: str,
    #mime_type: str = "application/pdf",
    mime_type: str = "image/jpeg",
    print_prompt: bool = False,
    print_raw_response: bool = False,
) -> str:
    # Load file directly from Google Cloud Storage
    file_part = Part.from_uri(
        uri=file_uri,
        mime_type=mime_type,
    )

    # Load contents
    contents = [file_part, prompt]

    # Send to Gemini
    response = model.generate_content(
        contents, generation_config=generation_config, safety_settings=safety_settings
    )

    if print_prompt:
        print("-------Prompt--------")
        print_multimodal_prompt(contents)

    if print_raw_response:
        print("\n-------Raw Response--------")
        print(response)

    return response.text

## Entity Extraction

[Named Entity Extraction](https://en.wikipedia.org/wiki/Named-entity_recognition) is a technique of Natural Language Processing to identify specific fields and values from unstructured text. For example, you can find key-value pairs from a filled out form, or get all of the important data from an invoice categorized by the type.

### Extract entities from an invoice

In this example, you will use a sample invoice and get all of the information in JSON format.

This is the prompt to be sent to Gemini along with the PDF document. Feel free to edit this for your specific use case.

## Document Classification

Document classification is the process for identifying the type of document. For example, invoice, W-2, receipt, etc.

In this example, you will use a sample tax form (W-2) and get the specific type of document from a specified list.

In [9]:
classification_prompt = """You are a document classification assistant. Given a document, your task is to find which category the document belongs to from the list of document categories provided below.

 Production_request
 Request_for_material_exchange

Which category does the above document belong to? Answer with one of the predefined document categories only. Answer in form "category:category name"
"""

You can see that Gemini successfully categorized the document.

### Chaining Classification and Extraction

These techniques can also be chained together to extract any number of document types. For example, if you have multiple types of documents to process, you can send each document to Gemini with a classification prompt, then based on that output, you can write logic to decide which extraction prompt to use.

In [50]:
generic_document_prompt = """You are a document entity extraction specialist. Most important identificator is "Request_number" which always presented over the key. Given a document, your task is to extract the text value of the following entities:

{}

- The JSON schema must be followed during the extraction.
- The values must only include text found in the document
- Do not normalize any entity value.
- If an entity is not found in the document, set the entity value to null.
"""

production_request_prompt = generic_document_prompt.format(
    """
{
  "Date": "",
  "Изделие": "",
  "Название_запроса": "",
  "Обозначение_ДСЕ": "",
  "Номер_чертежа": "",
  "Номер_служебной_записки": "",
  "Номер_техн_паспорта": "",
  "Номер_операции": "",
  "Содержание": "",
  "Author": "",
  "Head_of_technical_bureau": "",
  "Head_of_BTK": "",
  "Foreman": "",
  "Date_of_Author_Signature": "",
  "Request_number": "",
  "Расцеховка": "",
  "Master": "",
  "Change_in_EKD": "",
  "Номер_извещения": "",
  "Изменение_массы": "",
  "Решение": "",
  "Наименование": "",
  "Обозначение": "",
  "Количество": "",
  "Изготовлено": "",
  "Срок": "",
  "Изготовить_до": "",
  "Конструктор": "",
  "Начальник_бригады": "",
  "Прочность": "",
  "ТО_ОГТ": "",
  "Ведущий_конструктор": "",
  "Зам_нач_оп": "",
  "615-ПЗ": ""
}
    """
)

request_for_material_exchange = generic_document_prompt.format(
    """
{
  "Дата": "",
  "Изделие": "",
  "Название_запроса": "",
  "Request_number": "",
  "Расцеховка": "",
  "Номер_служебной_записки_извещения": "",
  "Обозначение_ДСЕ" : "",
  "Прошение_на_замену_материала": "",
  "Решение_на_замену_материала": "",
  "Технолог": "",
  "Начальник_бюро": ""
  "Конструктор": "",
  "Начальник_бригады": "",
  "Нач_бр_проч": "",
}
    """
)


# Map classification types to extraction prompts
classification_to_prompt = {
    "Production_request": production_request_prompt,
    "Request_for_material_exchange": request_for_material_exchange
}

In [51]:
import json
import time
from collections import deque


# Define the rate limit (requests per minute)
RATE_LIMIT = 2  # Adjust this to 2 for 2 documents per minute

# Initialize a queue to track requests and the timestamp of the last request
request_queue = deque(maxlen=RATE_LIMIT)
last_request_timestamp = time.time()  # Initialize with current time


def send_request_with_rate_limit(prompt, gcs_uri, print_prompt=False):
    """Sends a request to Gemini with rate limiting, handling potential errors.

    Args:
        prompt (str): The prompt to send to Gemini.
        gcs_uri (str): The URI of the Google Cloud Storage file.
        print_prompt (bool, optional): Whether to print the prompt. Defaults to True.

    Returns:
        str: The response text from Gemini, or None if an error occurs.
    """
    global last_request_timestamp  # Додали цей рядок

    # Check if queue is empty (for the first iteration)
    if not request_queue:
        last_request_timestamp = time.time()  # Update timestamp on first request

    # Calculate sleep time (even if 0)
    current_time = time.time()
    sleep_time = (last_request_timestamp + (60 / RATE_LIMIT)) - current_time
    if sleep_time > 0:
        time.sleep(sleep_time)
    else:
        time.sleep(0.01)  # Sleep for a minimal amount to avoid busy waiting

    # Update the timestamp for the last request
    last_request_timestamp = current_time

    # Add the request to the queue and send it
    request_queue.append((current_time, prompt, gcs_uri, print_prompt))

    # Send request to your process_document function (logic unchanged)
    try:
        response_text = process_document(prompt, gcs_uri, print_prompt=print_prompt).strip()
        return response_text
    except Exception as e:  # Catch generic exceptions for process_document
        print(f"Error processing document: {gcs_uri}")
        print(f"Exception: {e}")
        # Consider logging the error for further analysis and returning a default value
        return None




gcs_uris = uris
total_results = []
for gcs_uri in gcs_uris:
    print(f"\nFile: {gcs_uri}\n")

    if gcs_uri.endswith(".jpg") or gcs_uri.endswith(".tif") or gcs_uri.endswith(".TIF"):
        try:
            # Send to Gemini with Classification Prompt with rate limiting
            doc_classification = send_request_with_rate_limit(classification_prompt, gcs_uri).strip()
            dictionary = json.loads(doc_classification)
            print(f"Document Classification: {dictionary}")
        except Exception as e:  # Catch generic exceptions for classification processing
            print(f"Error processing document {gcs_uri} for classification:")
            print(f"Exception: {e}")
            continue  # Skip to the next file if an error occurs


        # Get Extraction prompt based on Classification
        if "document_category" in dictionary:
            extraction_prompt = classification_to_prompt.get(dictionary['document_category'])
        elif "category" in dictionary:
            extraction_prompt = classification_to_prompt.get(dictionary['category'])
        elif "answer" in dictionary:
            extraction_prompt = classification_to_prompt.get(dictionary['answer'])
        else:
            print(f"Document does not belong to a specified classes {gcs_uri}")
            # Download a PDF from Google Cloud Storage
            ! gsutil cp {gcs_uri} ./Pdf/

        # Створіть унікальне ім'я файлу на основі URI
        filename = gcs_uri.split("/")[-1].split(".")[0] + ".json"

        # Send to Gemini with Extraction Prompt with rate limiting
        try:
            extraction_response_text = send_request_with_rate_limit(extraction_prompt, gcs_uri, print_prompt=True).strip()
            print("\n-------Extracted Entities--------")
            extracted_text = json.loads(extraction_response_text)
            print(extracted_text)
        except Exception as e:  # Catch generic exceptions for extraction processing
            print(f"Error processing document {gcs_uri} for extraction:")
            print(f"Exception: {e}")
            continue  # Skip to the next file if an error occurs

        #Save each file separately----------1-----
        extracted_text["URL"] = gcs_uri

        # Save file to json with the same name
        with open (filename, 'w', encoding='utf-8') as outfile:
            json.dump(extracted_text, outfile, ensure_ascii=False)

        #Save all files in one total.json-----------------2-------------
        #total_results.append(extracted_text)

# Зберегти всі результати в один файл total.json
#with open('total.json', 'w', encoding='utf-8') as outfile:
    #json.dump(total_results, outfile, ensure_ascii=False)


File: gs://anton_may_test/1119_001.jpg

Document Classification: {'category': 'Request_for_material_exchange'}
-------Prompt--------


You are a document entity extraction specialist. Most important identificator is "Request_number" which always presented over the key. Given a document, your task is to extract the text value of the following entities:


{
  "Дата": "",
  "Изделие": "",
  "Название_запроса": "",
  "Request_number": "",
  "Расцеховка": "",
  "Номер_служебной_записки_извещения": "",
  "Обозначение_ДСЕ" : "",
  "Прошение_на_замену_материала": "",
  "Решение_на_замену_материала": "",
  "Технолог": "",
  "Начальник_бюро": ""
  "Конструктор": "",
  "Начальник_бригады": "",
  "Нач_бр_проч": "",
}
    

- The JSON schema must be followed during the extraction.
- The values must only include text found in the document
- Do not normalize any entity value.
- If an entity is not found in the document, set the entity value to null.


-------Extracted Entities--------
{'Дата': '12.08.2015', 'Изделие': '1-132.96', 'Название_запроса': 'ЗАПРОС\nНА ЗАМЕНУ\nМАТЕРИАЛА', 'Request_number': 'M-CM/1119', 'Расцеховка': '02,УО'

You are a document entity extraction specialist. Most important identificator is "Request_number" which always presented over the key. Given a document, your task is to extract the text value of the following entities:


{
  "Дата": "",
  "Изделие": "",
  "Название_запроса": "",
  "Request_number": "",
  "Расцеховка": "",
  "Номер_служебной_записки_извещения": "",
  "Обозначение_ДСЕ" : "",
  "Прошение_на_замену_материала": "",
  "Решение_на_замену_материала": "",
  "Технолог": "",
  "Начальник_бюро": ""
  "Конструктор": "",
  "Начальник_бригады": "",
  "Нач_бр_проч": "",
}
    

- The JSON schema must be followed during the extraction.
- The values must only include text found in the document
- Do not normalize any entity value.
- If an entity is not found in the document, set the entity value to null.


-------Extracted Entities--------
{'Дата': '18.08.15', 'Изделие': '132\n1-132.96', 'Название_запроса': 'ЗАПРОС\nНА ЗАМЕНУ\nМАТЕРИАЛА', 'Request_number': 'M-CM/1122', 'Расцеховка': None

You are a document entity extraction specialist. Most important identificator is "Request_number" which always presented over the key. Given a document, your task is to extract the text value of the following entities:


{
  "Дата": "",
  "Изделие": "",
  "Название_запроса": "",
  "Request_number": "",
  "Расцеховка": "",
  "Номер_служебной_записки_извещения": "",
  "Обозначение_ДСЕ" : "",
  "Прошение_на_замену_материала": "",
  "Решение_на_замену_материала": "",
  "Технолог": "",
  "Начальник_бюро": ""
  "Конструктор": "",
  "Начальник_бригады": "",
  "Нач_бр_проч": "",
}
    

- The JSON schema must be followed during the extraction.
- The values must only include text found in the document
- Do not normalize any entity value.
- If an entity is not found in the document, set the entity value to null.


-------Extracted Entities--------
{'Дата': '9.10.2015', 'Изделие': None, 'Название_запроса': 'ЗАПРОС\nНА ЗАМЕНУ\nМАТЕРИАЛА', 'Request_number': 'M-CM/204', 'Расцеховка': '01,35,УО', 'Но

You are a document entity extraction specialist. Most important identificator is "Request_number" which always presented over the key. Given a document, your task is to extract the text value of the following entities:


{
  "Дата": "",
  "Изделие": "",
  "Название_запроса": "",
  "Request_number": "",
  "Расцеховка": "",
  "Номер_служебной_записки_извещения": "",
  "Обозначение_ДСЕ" : "",
  "Прошение_на_замену_материала": "",
  "Решение_на_замену_материала": "",
  "Технолог": "",
  "Начальник_бюро": ""
  "Конструктор": "",
  "Начальник_бригады": "",
  "Нач_бр_проч": "",
}
    

- The JSON schema must be followed during the extraction.
- The values must only include text found in the document
- Do not normalize any entity value.
- If an entity is not found in the document, set the entity value to null.


-------Extracted Entities--------
{'Дата': '1.10.15', 'Изделие': '1-132.96', 'Название_запроса': 'ЗАПРОС\nНА ЗАМЕНУ\nМАТЕРИАЛА', 'Request_number': None, 'Расцеховка': '02,УО', 'Номер_с

...

You are a document entity extraction specialist. Most important identificator is "Request_number" which always presented over the key. Given a document, your task is to extract the text value of the following entities:


{
  "Date": "",
  "Изделие": "",
  "Название_запроса": "",
  "Обозначение_ДСЕ": "",
  "Номер_чертежа": "",
  "Номер_служебной_записки": "",
  "Номер_техн_паспорта": "",
  "Номер_операции": "",
  "Содержание": "",
  "Author": "",
  "Head_of_technical_bureau": "",
  "Head_of_BTK": "",
  "Foreman": "",
  "Date_of_Author_Signature": "",
  "Request_No": "",
  "Расцеховка": "",
  "Master": "",
  "Change_in_EKD": "",
  "Номер_извещения": "",
  "Изменение_массы": "",
  "Решение": "",
  "Наименование": "",
  "Обозначение": "",
  "Количество": "",
  "Изготовлено": "",
  "Срок": "",
  "Изготовить_до": "",
  "Конструктор": "",
  "Начальник_бригады": "",
  "Прочность": "",
  "ТО_ОГТ": "",
  "Ведущий_конструктор": "",
  "Зам_нач_оп": "",
  "615-ПЗ": ""
}
    

- The JSON schema must

You are a document entity extraction specialist. Most important identificator is "Request_number" which always presented over the key. Given a document, your task is to extract the text value of the following entities:


{
  "Дата": "",
  "Изделие": "",
  "Название_запроса": "",
  "Request_number": "",
  "Расцеховка": "",
  "Номер_служебной_записки_извещения": "",
  "Обозначение_ДСЕ" : "",
  "Прошение_на_замену_материала": "",
  "Решение_на_замену_материала": "",
  "Технолог": "",
  "Начальник_бюро": ""
  "Конструктор": "",
  "Начальник_бригады": "",
  "Нач_бр_проч": "",
}
    

- The JSON schema must be followed during the extraction.
- The values must only include text found in the document
- Do not normalize any entity value.
- If an entity is not found in the document, set the entity value to null.


-------Extracted Entities--------
{'Дата': '17 02-04', 'Изделие': '1-132.96', 'Название_запроса': 'ЗАПРОС\nНА ЗАМЕНУ\nМАТЕРИАЛА', 'Request_number': 'M-CM/223', 'Расцеховка': None, 'Ном

You are a document entity extraction specialist. Most important identificator is "Request_number" which always presented over the key. Given a document, your task is to extract the text value of the following entities:


{
  "Date": "",
  "Изделие": "",
  "Название_запроса": "",
  "Обозначение_ДСЕ": "",
  "Номер_чертежа": "",
  "Номер_служебной_записки": "",
  "Номер_техн_паспорта": "",
  "Номер_операции": "",
  "Содержание": "",
  "Author": "",
  "Head_of_technical_bureau": "",
  "Head_of_BTK": "",
  "Foreman": "",
  "Date_of_Author_Signature": "",
  "Request_No": "",
  "Расцеховка": "",
  "Master": "",
  "Change_in_EKD": "",
  "Номер_извещения": "",
  "Изменение_массы": "",
  "Решение": "",
  "Наименование": "",
  "Обозначение": "",
  "Количество": "",
  "Изготовлено": "",
  "Срок": "",
  "Изготовить_до": "",
  "Конструктор": "",
  "Начальник_бригады": "",
  "Прочность": "",
  "ТО_ОГТ": "",
  "Ведущий_конструктор": "",
  "Зам_нач_оп": "",
  "615-ПЗ": ""
}
    

- The JSON schema must

You are a document entity extraction specialist. Most important identificator is "Request_number" which always presented over the key. Given a document, your task is to extract the text value of the following entities:


{
  "Date": "",
  "Изделие": "",
  "Название_запроса": "",
  "Обозначение_ДСЕ": "",
  "Номер_чертежа": "",
  "Номер_служебной_записки": "",
  "Номер_техн_паспорта": "",
  "Номер_операции": "",
  "Содержание": "",
  "Author": "",
  "Head_of_technical_bureau": "",
  "Head_of_BTK": "",
  "Foreman": "",
  "Date_of_Author_Signature": "",
  "Request_No": "",
  "Расцеховка": "",
  "Master": "",
  "Change_in_EKD": "",
  "Номер_извещения": "",
  "Изменение_массы": "",
  "Решение": "",
  "Наименование": "",
  "Обозначение": "",
  "Количество": "",
  "Изготовлено": "",
  "Срок": "",
  "Изготовить_до": "",
  "Конструктор": "",
  "Начальник_бригады": "",
  "Прочность": "",
  "ТО_ОГТ": "",
  "Ведущий_конструктор": "",
  "Зам_нач_оп": "",
  "615-ПЗ": ""
}
    

- The JSON schema must

You are a document entity extraction specialist. Most important identificator is "Request_number" which always presented over the key. Given a document, your task is to extract the text value of the following entities:


{
  "Date": "",
  "Изделие": "",
  "Название_запроса": "",
  "Обозначение_ДСЕ": "",
  "Номер_чертежа": "",
  "Номер_служебной_записки": "",
  "Номер_техн_паспорта": "",
  "Номер_операции": "",
  "Содержание": "",
  "Author": "",
  "Head_of_technical_bureau": "",
  "Head_of_BTK": "",
  "Foreman": "",
  "Date_of_Author_Signature": "",
  "Request_No": "",
  "Расцеховка": "",
  "Master": "",
  "Change_in_EKD": "",
  "Номер_извещения": "",
  "Изменение_массы": "",
  "Решение": "",
  "Наименование": "",
  "Обозначение": "",
  "Количество": "",
  "Изготовлено": "",
  "Срок": "",
  "Изготовить_до": "",
  "Конструктор": "",
  "Начальник_бригады": "",
  "Прочность": "",
  "ТО_ОГТ": "",
  "Ведущий_конструктор": "",
  "Зам_нач_оп": "",
  "615-ПЗ": ""
}
    

- The JSON schema must

 Об'єднання всіх json у один файл total.json (Зручно для подальшої інтеграції із Bigquery)

In [52]:
import json
import os

def unite_json_files_to_jsonl(input_folder, output_file):
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for filename in os.listdir(input_folder):
            if filename.endswith('.json'):
                file_path = os.path.join(input_folder, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        data = json.load(file)
                        # Кожний об'єкт або елемент з data записуємо у новий рядок
                        if isinstance(data, list):
                            for item in data:
                                outfile.write(json.dumps(item, ensure_ascii=False) + '\n')
                        else:
                            outfile.write(json.dumps(data, ensure_ascii=False) + '\n')
                except json.JSONDecodeError as e:
                    print(f'Помилка декодування JSON у файлі {file_path}: {e}')
                except Exception as e:
                    print(f'Інша помилка при обробці файлу {file_path}: {e}')

    print(f'Дані об\'єднані у форматі JSONL в файл {output_file}')

# Вкажи шлях до папки з файлами JSON та ім'я вихідного файлу
input_folder = '.'
output_file = 'total.json'

unite_json_files_to_jsonl(input_folder, output_file)

Помилка декодування JSON у файлі ./total.json: Extra data: line 2 column 1 (char 920)
Дані об'єднані у форматі JSONL в файл total.json


## Document Question Answering

Gemini can be used to answer questions about a document.

This example answers a question about eligibility for the Family and Medical Leave Act (FMLA).

In [ ]:
qa_prompt = """Today is December 31, 2023. Arthur started at the company on April 1, 2023, and he works 40 hours a week. Is Arthur eligible for FMLA? Give me the answer first, followed by an explanation."""

In [ ]:
# Send Q&A Prompt to Gemini
response_text = process_document(
    qa_prompt,
    "gs://cloud-samples-data/generative-ai/pdf/family and_medical_leave_act.pdf",
)

print(f"Answer: {response_text}")

## Document Summarization

Gemini can also be used to summarize or paraphrase a document's contents. Your prompt can specify how detailed the summary should be or specific formatting, such as bullet points or paragraphs.

In [ ]:
summarization_prompt = """You are a very professional document summarization specialist. Given a document, your task is to provide a detailed summary of the content of the document.

If it includes images, provide descriptions of the images.
If it includes tables, extract all elements of the tables.
If it includes graphs, explain the findings in the graphs.
Do not include any numbers that are not mentioned in the document.
"""

In [ ]:
# Send Summarization Prompt to Gemini
response_text = process_document(
    summarization_prompt,
    "gs://cloud-samples-data/generative-ai/pdf/fdic_board_meeting.pdf",
)

print(f"Summarization: {response_text}")

## Table parsing from documents

Gemini can parse contents of a table and return it in a structured format, such as HTML or markdown.

In [ ]:
table_extraction_prompt = """What is the html code of the table in this document?"""

In [ ]:
# Send Table Extraction Prompt to Gemini
response_text = process_document(
    table_extraction_prompt,
    "gs://cloud-samples-data/generative-ai/pdf/salary_table.pdf",
)

print(response_text)

## Document Translation

Gemini can translate documents between languages. This example translates meeting notes from English into French and Spanish.

In [ ]:
translation_prompt = """Translate the first paragraph into French and Spanish. Label each paragraph with the target language."""

In [ ]:
# Send Translation Prompt to Gemini
response_text = process_document(
    translation_prompt,
    "gs://cloud-samples-data/generative-ai/pdf/fdic_board_meeting.pdf",
)

print(response_text)

## Document Comparsion

Gemini can compare and contrast the contents of multiple documents. This example finds the changes in the IRS Form 1040 between 2013 and 2023.

Note: when working with multiple documents, the order can matter and should be specified in your prompt.

In [ ]:
comparsion_prompt = """The first document is from 2013, the second one from 2023. How did the standard deduction evolve?"""

In [ ]:
# Send Comparsion Prompt to Gemini
file_part1 = Part.from_uri(
    uri="gs://cloud-samples-data/generative-ai/pdf/form_1040_2013.pdf",
    mime_type="application/pdf",
)

file_part2 = Part.from_uri(
    uri="gs://cloud-samples-data/generative-ai/pdf/form_1040_2023.pdf",
    mime_type="application/pdf",
)

# Load contents
contents = [file_part1, file_part2, comparsion_prompt]

# Send to Gemini
response = model.generate_content(
    contents, generation_config=generation_config, safety_settings=safety_settings
)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("-------Output--------")
print(response.text)